Source: [NYC Taxi and Limousine Commission (TLC)](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

[Data Dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

### Overwiew
The primary objective of this project is to build a regression model that predicts taxi cab demand by time by location.


**The isuue: demand in taxi / car share industry**
- Drivers vs Customers (Supply vs Demand)
- Companies need to forecast demand

**Solution: Predict taxi demand per hour by location**

Operations team can adjust the distribution of drivers (send drivers to high demand areas)

### Project Steps
1. Load dataset
2. Explore dataset for any unusual values
3. Conduct a complete exploratory data analysis
4. Perform any data cleaning and data analysis steps to understand unusual variables (e.g., outliers)
5. Use descriptive statistics to learn more about the data
6. Create and run a regression model

## 1. Loading, inspecting and validating the data

In [1]:
import pandas as pd
from pathlib import Path
import requests
# %pip install pyarrow

In [2]:
def get_taxi_data(year: int, month: int) -> str:
    """Get TLC data"""
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02}.parquet"
    response = requests.get(url)

    if response.status_code == 200:
        path = f"../data/raw/rides_{year}_{month:02}.parquet"
        with open(path, 'wb') as file:
            file.write(response.content)
        return path
    else:   
        raise RuntimeError(f"Failed to download {url}")

In [3]:
def merge_taxi_data(start_year: int, end_year: int) -> pd.DataFrame:
    """Merge TLC data. 
    The function is writen with concideration 
    that I might decide use more data in the future"""
    df = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            file_path = get_taxi_data(year, month)
            df_month = pd.read_parquet(file_path)
            df.append(df_month)
    merged_data = pd.concat(df, ignore_index=True)
    return merged_data

In [4]:
# Range of years for which to download and merge the data. Set for now for 2023 only
start_year = 2023
end_year = 2023

merged_data = merge_taxi_data(start_year, end_year)
merged_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,NaN


In [5]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38310226 entries, 0 to 38310225
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee         

In [8]:
merged_data.isnull().sum() / len(merged_data) * 100

VendorID                  0.000000
tpep_pickup_datetime      0.000000
tpep_dropoff_datetime     0.000000
passenger_count           3.417772
trip_distance             0.000000
RatecodeID                3.417772
store_and_fwd_flag        3.417772
PULocationID              0.000000
DOLocationID              0.000000
payment_type              0.000000
fare_amount               0.000000
extra                     0.000000
mta_tax                   0.000000
tip_amount                0.000000
tolls_amount              0.000000
improvement_surcharge     0.000000
total_amount              0.000000
congestion_surcharge      3.417772
airport_fee              92.182184
Airport_fee              11.235588
dtype: float64

**Observations and insights**

Out of 38,310,226 columns there are a few columns with a low number of missing values (less than 3,5%):

| Field Name | Description |
| -------- | ------- |
| Passenger_count | The number of passengers in the vehicle. **This is a driver-entered value**|
| RateCodeID |The final rate code in effect at the end of the trip. **1= Standard rate;  2=JFK; 3=Newark; 4=Nassau or Westchester; 5=Negotiated fare; 6=Group ride** |
| Store_and_fwd_flag | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. **Y= store and forward trip; N= not a store and forward trip** |
| Congestion_Surcharge | Total amount collected in trip for NYS congestion surcharge |
| Airport_fee | $1.25 for pick up only at LaGuardia and John F. Kennedy Airports |

There are also two airport_fee columns that require further investigation (and probably merging).

In [6]:
merged_data.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
count,3.831023e+07,38310226,38310226,3.700087e+07,3.831023e+07,3.700087e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.831023e+07,3.700087e+07,2.995023e+06,3.400585e+07
mean,1.738894e+00,2023-07-02 22:47:58.448232,2023-07-02 23:04:41.459181,1.370426e+00,4.088946e+00,1.641906e+00,1.651755e+02,1.639500e+02,1.184585e+00,1.952225e+01,1.556057e+00,4.856167e-01,3.522265e+00,5.897361e-01,9.794487e-01,2.846194e+01,2.264610e+00,1.074086e-01,1.429429e-01
min,1.000000e+00,2001-01-01 00:06:49,1970-01-20 10:16:32,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.087300e+03,-3.917000e+01,-5.000000e-01,-4.110000e+02,-9.130000e+01,-1.000000e+00,-1.094050e+03,-2.500000e+00,-1.250000e+00,-1.750000e+00
25%,1.000000e+00,2023-04-02 18:25:17,2023-04-02 18:43:06.250000,1.000000e+00,1.040000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.595000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,2023-06-27 18:43:55,2023-06-27 18:59:09,1.000000e+00,1.790000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.100000e+01,2.500000e+00,0.000000e+00,0.000000e+00
75%,2.000000e+00,2023-10-06 21:55:14,2023-10-06 22:11:37,1.000000e+00,3.400000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.190000e+01,2.500000e+00,5.000000e-01,4.410000e+00,0.000000e+00,1.000000e+00,3.072000e+01,2.500000e+00,0.000000e+00,0.000000e+00
max,6.000000e+00,2024-01-03 19:42:57,2024-01-03 20:15:55,9.000000e+00,3.457294e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,3.869836e+05,1.000250e+04,5.316000e+01,4.174000e+03,6.655600e+02,1.000000e+00,3.869876e+05,2.750000e+00,1.250000e+00,1.750000e+00
std,4.443591e-01,NaN,NaN,8.925924e-01,2.412509e+02,7.427351e+00,6.399765e+01,6.985543e+01,5.562740e-01,7.572729e+01,2.450399e+00,1.095629e-01,4.147060e+00,2.200999e+00,1.991462e-01,7.712821e+01,7.971481e-01,3.556511e-01,4.772075e-01


## 2. Understanding the data - investigating the variables

In [7]:
df_sort = merged_data.sort_values(by=['trip_distance'],ascending=False)
df_sort.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
25178451,2,2023-08-15 04:02:00,2023-08-15 04:13:00,NaN,345729.44,NaN,None,100,263,0,13.27,0.0,0.5,0.00,0.00,1.0,17.27,NaN,NaN,NaN
16115613,2,2023-05-11 09:30:00,2023-05-11 09:57:00,NaN,342344.85,NaN,None,116,239,0,-0.11,0.0,0.5,0.00,0.00,1.0,3.89,NaN,NaN,NaN
5959075,2,2023-02-21 05:19:00,2023-02-21 05:36:00,NaN,335004.33,NaN,None,142,42,0,17.75,0.0,0.5,2.18,0.00,1.0,23.93,NaN,NaN,NaN
12672047,2,2023-04-30 18:37:00,2023-04-30 19:02:00,NaN,296881.42,NaN,None,233,151,0,28.39,0.0,0.5,6.48,0.00,1.0,38.87,NaN,NaN,NaN
28053740,2,2023-09-28 13:35:00,2023-09-28 14:53:00,NaN,265869.44,NaN,None,14,140,0,76.02,0.0,0.5,0.00,6.94,1.0,86.96,NaN,NaN,NaN
3046991,2,2023-01-23 17:14:00,2023-01-23 17:20:00,NaN,258928.15,NaN,None,238,239,0,13.65,0.0,0.5,3.53,0.00,1.0,21.18,NaN,NaN,NaN
25162910,2,2023-08-10 02:05:00,2023-08-10 02:17:00,NaN,232976.75,NaN,None,42,239,0,14.45,0.0,0.5,3.69,0.00,1.0,22.14,NaN,NaN,NaN
3009826,2,2023-01-07 18:18:00,2023-01-07 18:47:00,NaN,225987.37,NaN,None,261,263,0,56.49,0.0,0.5,12.10,0.00,1.0,72.59,NaN,NaN,NaN
9369796,2,2023-03-26 14:43:00,2023-03-26 15:12:00,NaN,216986.96,NaN,None,79,239,0,28.71,0.0,0.5,6.54,0.00,1.0,39.25,NaN,NaN,NaN
31550098,2,2023-10-24 08:15:00,2023-10-24 08:32:00,NaN,205544.17,NaN,None,229,236,0,23.46,0.0,0.5,3.74,0.00,1.0,28.70,NaN,NaN,NaN


**Observations and insights**

Longest rides are more than 345K miles - requres further investigation.

In [9]:
total_amount_sorted = merged_data.sort_values(['total_amount'], ascending=False)['total_amount']
total_amount_sorted.head(20)

17439335    386987.63
25349953    187513.90
25555738    143167.45
26249203     29156.90
27892250     12015.47
31549066      6339.00
14388207      6304.90
36222775      4269.16
9952970       2451.00
37232285      2372.79
3863236       2208.10
36892829      2203.14
7322951       2100.00
10767501      1968.75
30387473      1939.55
34846157      1814.27
33896694      1736.29
15284379      1715.95
23990259      1435.19
15996874      1327.52
Name: total_amount, dtype: float64

**Observations and insights**

Few top values are quitew high, with first three values being significantly higher than the others.

In [10]:
total_amount_sorted.tail(20)

15843478    -749.40
2849551     -751.00
19115608    -751.00
5054337     -757.55
35080508    -765.89
12568248    -769.60
25997487    -796.00
34028474    -801.00
12539166    -801.00
12539168    -807.55
29213930    -819.75
35271157    -830.94
26144088    -872.75
15584232    -900.50
31312868    -901.00
23942938    -901.00
37622420    -901.00
35784750    -907.09
6726648     -982.95
38123079   -1094.05
Name: total_amount, dtype: float64

In [11]:
merged_data['payment_type'].value_counts()

payment_type
1    29856932
2     6405059
0     1309356
4      498015
3      240862
5           2
Name: count, dtype: int64

**Observations and insights**

I assume that in the dataset, payment_type is encoded in the following integers:
0: Credit card
1: Cash
2: No charge
3: Dispute
4: Unknown
5: Voided trips

According to [Data Dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf) for Yellow Taxi Trip Records (updated in May 11 2022) the payment type shold be represented as following:

| Field Name | Description |
| -------- | ------- |
| Payment_type | A numeric code signifying how the passenger paid for the trip. **1= Credit card; 2= Cash; 3= No charge; 4= Dispute; 5= Unknown; 6= Voided trip** |

Investigate if 5 represent voided trip(6) and 0 represent cash. 

In [14]:
merged_data[merged_data['payment_type']==5]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
8050105,1,2023-03-20 00:21:17,2023-03-20 01:00:24,1.0,12.1,2.0,Y,132,137,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
16656441,1,2023-06-05 10:08:32,2023-06-05 10:09:58,1.0,0.0,1.0,N,50,50,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [29]:
# Is taximeter broken?!
merged_data[merged_data['payment_type']==0]\
[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'total_amount']]\
.sort_values(by=['trip_distance'],ascending=False)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,total_amount
25178451,2023-08-15 04:02:00,2023-08-15 04:13:00,345729.44,13.27,17.27
16115613,2023-05-11 09:30:00,2023-05-11 09:57:00,342344.85,-0.11,3.89
5959075,2023-02-21 05:19:00,2023-02-21 05:36:00,335004.33,17.75,23.93
12672047,2023-04-30 18:37:00,2023-04-30 19:02:00,296881.42,28.39,38.87
28053740,2023-09-28 13:35:00,2023-09-28 14:53:00,265869.44,76.02,86.96
...,...,...,...,...,...
34888457,2023-11-19 02:27:57,2023-11-19 02:38:24,0.00,11.59,15.59
34888456,2023-11-19 02:08:51,2023-11-19 02:21:13,0.00,21.50,25.50
31469679,2023-10-07 13:35:27,2023-10-07 14:07:26,0.00,35.72,39.72
34888454,2023-11-19 02:57:37,2023-11-19 03:18:03,0.00,32.32,36.32


In [15]:
# Average tip for trips paid for with credit card
avg_cc_tip = merged_data[merged_data['payment_type']==0]['tip_amount'].mean()
print('Avg. ccredit card tip:', avg_cc_tip)

# Average tip for trips paid for with cash
avg_cash_tip = merged_data[merged_data['payment_type']==1]['tip_amount'].mean()
print('Avg. cash tip:', avg_cash_tip)

Avg. ccredit card tip: 2.7739578235407314
Avg. cash tip: 4.396300929043871


In [16]:
merged_data['VendorID'].value_counts()

VendorID
2    28263844
1    10037714
6        8668
Name: count, dtype: int64

In [17]:
# Average amount paid for each vendor
merged_data.groupby(['VendorID']).mean(numeric_only=True)[['total_amount']]

,total_amount
VendorID,
1,27.228547
2,28.894853
6,45.157738


In [18]:
# Cash payment by passenger count
cash = merged_data[merged_data['payment_type']==1]
cash['passenger_count'].value_counts()

passenger_count
1.0    22698465
2.0     4421685
3.0     1071163
4.0      558370
0.0      456885
5.0      393234
6.0      256775
8.0         224
7.0          85
9.0          46
Name: count, dtype: int64

In [19]:
# Average tip amount for each passenger count (cash payments only)
cash.groupby(['passenger_count']).mean(numeric_only=True)[['tip_amount']]

,tip_amount
passenger_count,
0.0,3.892517
1.0,4.289000
2.0,4.858613
3.0,4.677780
4.0,5.008497
5.0,4.360620
6.0,4.355261
7.0,8.971765
8.0,12.779598
